In [1]:
# model.ipynb

import cv2
import numpy as np
from collections import deque
from tensorflow.keras.models import load_model

# --- Config ---
SEQUENCE_LENGTH = 15
IMG_SIZE = 224
CLASSES = ['bed', 'before', 'candy', 'cool', 'drink', 'go', 'help', 'thin']

# --- Load Model ---
model = load_model('asl_mobilenetv2_finetuned.h5')

# --- Initialize sequence buffer ---
sequence = deque(maxlen=SEQUENCE_LENGTH)


2025-04-08 20:39:10.901209: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# --- Start Webcam ---
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocess: resize and normalize
    frame_resized = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
    frame_rgb = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB)
    frame_normalized = frame_rgb / 255.0

    sequence.append(frame_normalized)

    # Only predict if sequence is full
    if len(sequence) == SEQUENCE_LENGTH:
        input_data = np.expand_dims(sequence[-1], axis=0)  # shape: (1, 224, 224, 3)
        prediction = model.predict(input_data, verbose=0)[0]
        predicted_label = CLASSES[np.argmax(prediction)]
        confidence = np.max(prediction)


        # Show prediction on frame
        cv2.putText(frame, f"{predicted_label} ({confidence:.2f})", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Show frame
    cv2.imshow('ASL Real-Time Prediction', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()

2025-04-08 20:39:29.873 Python[11393:572670] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
2025-04-08 20:39:31.897 Python[11393:572670] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


: 